In [742]:
from random import *
import hashlib

def puissance (a,e,n):
  p = 1
  while e>0:
    if (e % 2 != 0):
      p=(p*a)%n
    a=(a*a)%n
    e=e//2
  return p

In [743]:
# Test puissance
def test_puissance():
    assert(puissance(4,2,5)==1)
test_puissance()

In [744]:
def test_premier_Fermat(n): # test de Fermat
   if ( (puissance(2,n-1,n)==1) and
      (puissance(3,n-1,n)==1) and
      (puissance(5,n-1,n)==1) and
      (puissance(7,n-1,n)==1) and
      (puissance(11,n-1,n)==1) and
      (puissance(13,n-1,n)==1) ):
      return True; # probablement premier (garantie si n<2^15)
   return False

In [745]:
# Test premier de Fermat
def test_Fermat():
    assert(test_premier_Fermat(22)==False)
    assert(test_premier_Fermat(647)==True)
test_Fermat()

In [746]:
def temoin_Miller(n, a):
    s = 0
    d = n-1
    while d % 2 == 0:
       d = d//2
    x = puissance(a, d, n)
    if(x == 1 or x == n-1):
        return False
    for i in range(s-1):
        x = puissance(x, 2, n)
        if(x == n-1):
            return False
    return True

In [747]:
# Test témoin de Miller 
def test_temoin_Miller():
    assert(temoin_Miller(3,9)==True)
    assert(temoin_Miller(4,9)==False)
test_temoin_Miller()

In [748]:
# Le GMP recommande k=25 pour avoir une proba d'erreur < 2**-50
def test_premier_Miller_Rabin(n, k):
    if n == 2 or n == 3:
       return True
    if n % 2 == 0:
       return False
    for i in range(k):
        a = randint(2, n-2)
        if(temoin_Miller(n, a)):
            return False
    return True

In [749]:
# Test de primalité de Miller Rabin
def test_premier_Miller():
    assert(test_premier_Miller_Rabin(3,1)==True)
    assert(test_premier_Miller_Rabin(42,5)==False)
test_premier_Miller()

In [750]:
def pgcd(u, v):
  t = 0
  while (v):
    t = u
    u = v
    v = t % v
  if(u<0):
    return -u
  else:
    return u

In [751]:
# Test PGCD
def test_pgcd():
    assert(pgcd(5,4)==1)
    assert(pgcd(222,448)==2)
test_pgcd()

In [752]:
def bezout(a, b):

  # On sauvegarde les valeurs de a et b.
   a0 = a
   b0 = b

  # On laisse invariant p*a0 + q*b0 = a et  r*a0 + s*b0 = b.
   p = 1
   q = 0
   r = 0
   s = 1
   c = 1
   quotient = 1
   nouveau_r = 1
   nouveau_s = 1

  # La boucle principale.
   while (b != 0):
    c = a % b
    quotient = a//b
    a = b
    b = c
    nouveau_r = p - (quotient * r)
    nouveau_s = q - (quotient * s)
    p = r
    q = s
    r = nouveau_r
    s = nouveau_s
   if p < 0: # Si p est négatif retourne p+phi
     p = p+b0
   return p  # on n'a besoin que de p 

In [753]:
# Test théorème de Bezout
def test_Bezout():
    assert(bezout(50,47)==16)
    assert(bezout(15,5)==0)
    assert(bezout(445,457)==38)
test_Bezout()

In [754]:
def generateurCle(c,b):
  p=randint(c,b)
  q=randint(c,b)
  e=randint(c,b)
  d=randint(c,b)

  while test_premier_Miller_Rabin(p,25)==False: 
    p = randint(c,b)
  while test_premier_Miller_Rabin(q,25)==False:
    q = randint(c,b)

  n=p*q
  
  phi = (p - 1)*(q - 1)
 
  while 1>=e>=phi or pgcd(e,phi)!=1 :
      e = randint(c,b)
  d= int(bezout(int(e),int(phi)))
  return e,d,n 

In [755]:
def chiffrer(message,clé,n):
    return puissance(message,clé,n)

In [756]:
# Test de la fonction chiffrer
def test_chiffrer():
    assert(chiffrer(1234,41,2449)==1927)
test_chiffrer()

In [757]:
def dechiffrer(messageChiffre,clé,n):
    return puissance(messageChiffre,clé,n)

In [758]:
# Test de la fonction dechiffrer 
def test_dechiffrer():
    assert(dechiffrer(1927,1541,2449)==1234)
test_dechiffrer()

In [759]:
def str_to_int(m):
    s=0
    b=1
    for i in range(len(m)):
        s=s+ord(m[i])*b
        b=b*256
    return s

In [760]:
def test_str_to_int():
    assert(str_to_int('a')==97)
    assert(str_to_int('k')==107)
test_str_to_int()

In [761]:
def int_to_str(c):
    s =""
    q,r= divmod(c,256)
    s=s+str(chr(r))
    while q != 0:
        q,r = divmod(q,256)
        s =s +str(chr(r))
    return s 


In [762]:
def test_int_to_str():
    assert(int_to_str(97)=='a')
    assert(int_to_str(107)=='k')
test_int_to_str()

In [763]:
def calculEmpreinte(message): 
    m=hashlib.sha256(repr(message).encode())
    return m.hexdigest()

In [764]:
def decoupe(variable,taille,liste,strVariable):
    decoupeIndex = 0
    for i in range(1,len(variable)):
        if(variable[i]!='0' and i%taille==0):
            liste.append(int(variable[decoupeIndex:i]))
            strVariable+=str(int(variable[decoupeIndex:i]))
            decoupeIndex=i
            
        if(i==len(variable)-1):
            liste.append(int(variable[decoupeIndex:len(variable)]))
            strVariable+=str(int(variable[decoupeIndex:len(variable)]))

In [765]:
# Test de la fonction de découpe d'un message 
def test_decoupe():
    l=[]
    strL = ''
    liste=[]
    strListe = '123456'
    liste.append(12)
    liste.append(34)
    liste.append(56)
    decoupe(strListe,2,l,strL)
    assert(l== liste)
    
test_decoupe()

In [766]:
def chiffrerEnsemble(liste, cle):
       for i in range(len(liste)):
            liste[i]=chiffrer(liste[i],cle[0],cle[1])

In [767]:
# Test de la fonction chiffrer ensemble 
def test_chiffrerEnsemble():
    liste = []
    liste.append(12)
    liste.append(34)
    liste.append(56)
    gC=generateurCle(10**1,10**2)
    clé= [gC[0],gC[2]]
    chiffrerEnsemble(liste,clé)
    l =[]
    l.append(chiffrer(12,clé[0],clé[1]))
    assert(l==liste)

In [768]:
def chiffrement_Asymetrique_Signe(message, clePrivee, clePublique):
    strEmpreinte = ""
    strEmpreinteC = ""
    messageEmpreinte = []
    messageParti = []
    strMessage = ""

    #Verifie si le message est un string, si oui le convertit en int
    if(isinstance(message, str)):
        message = str_to_int(message)

    #Calcule de l'empreinte du message
    empreinte = str(str_to_int(calculEmpreinte(message)))
    tailleMessage = int(len(str(message))/(len(str(message))/2))
    tailleEmpreinte = int(len(empreinte)/(len(empreinte)/2))

    #Découpe de l'empreinte pour pouvoir mieux la chiffrer
    decoupe(empreinte, tailleEmpreinte, messageEmpreinte, strEmpreinte)
    print("Empreinte: "+empreinte)

    #On decoupe le message en plusieurs morceaux
    decoupe(str(message), tailleMessage, messageParti, strMessage)

    #Chiffrage de l'empreinte avec la clé privée de l'envoyeur
    for i in range(0, len(messageEmpreinte)):
        messageEmpreinte[i] = chiffrer(
            messageEmpreinte[i], clePrivee[0], clePrivee[1])
        strEmpreinteC += str(messageEmpreinte[i])
    print("Empreinte Chiffré: "+strEmpreinteC)

    #Concaténation du message et de l'empreinte chiffrée
    print("Message + empreinte: "+strMessage+strEmpreinteC)

    #On chiffre l'ensemble des empreintes et des messages
    chiffrerEnsemble(messageEmpreinte, clePublique)
    chiffrerEnsemble(messageParti, clePublique)

    #Envoie du message finale chiffrée avec la clé publique du receveur
    return messageParti, messageEmpreinte



In [769]:
def verif_signature(messageParti, messageEmpreinte, clePrivee, clePublique):
    messageEmpreinteDecryptage = messageEmpreinte
    messagePartiDecryptage = messageParti
    empreinte1 = ""
    empreinteDecrypt = ""
    message = ""

    #Dechiffrage de l'empreinte
    for i in range(len(messageEmpreinteDecryptage)):
      messageEmpreinteDecryptage[i] = dechiffrer(
          messageEmpreinteDecryptage[i], clePrivee[0], clePrivee[1])
      empreinte1 += str(messageEmpreinteDecryptage[i])

    #Dechiffrage du message
    for i in range(len(messagePartiDecryptage)):
      messagePartiDecryptage[i] = dechiffrer(
          messagePartiDecryptage[i], clePrivee[0], clePrivee[1])
      message += str(messagePartiDecryptage[i])

    print("Message déchiffré: "+message+empreinte1)
    print("Message : "+message)
    print("Empreinte du message chiffrée: "+empreinte1)

    #Déchiffrage de l'empreinte avec la clé publique
    for i in range(len(messageEmpreinteDecryptage)):
      messageEmpreinteDecryptage[i] = str(dechiffrer(
          messageEmpreinteDecryptage[i], clePublique[0], clePublique[1]))
      empreinteDecrypt += str(messageEmpreinteDecryptage[i])

    #Vérification de l'empreinte déchiffrer, il faut qu'elle corresponde à l'empreinte du message
    if(str_to_int(calculEmpreinte(int(message)))) == (int(empreinteDecrypt)):
       return int(message)
    else:
       return -1
    

In [770]:
# Test de l'envoi et de la reception d'un message
def test_envoiEtReceptionMessage():
    message = 1234
    gA = generateurCle(10**38, 10**39)
    clePublique = [gA[0], gA[2]]
    clePrivee = [gA[1], gA[2]]
    messageFinal = chiffrement_Asymetrique_Signe(
        message, clePrivee, clePublique)
    messageVerif = verif_signature(
        messageFinal[0], messageFinal[1], clePrivee, clePublique)
    assert(messageVerif == message)


test_envoiEtReceptionMessage()


Empreinte: 2744372168965420703255761072842808216329619802047789678765124104220163271391731439411469307863996541660886798191291340339988553483486636194939326959792944
Empreinte Chiffré: 30822061916433491289727918293396265737861799048347049684682048324329389026865310337644161462503344985924065104665854904468057119591374849664089183390908677746772329050992641084533400577834227851875898863042800701614229280193348000901678177093470238077140022930734769466621592484954094023359601739248111406785352750007148551669422024703513543391172402818912472656464602951282820510448658072225102415488958943939577519083575712166224044774975842627514696584060123954602416502374408486227081565437576827730773502242131526555008464036968044152133151650843563872230738971215778493540598489015402210710397858341415446687257286211653712861399135295600025854252903237087732542087466260686943257482334966780892321878017786757062451941485386220540986934774901027729978807497216930070713954263299031887381771605696749600328772

In [771]:
#Alice
gA = generateurCle(10**38, 10**39)
clePubliqueA = [gA[0], gA[2]]
clePriveeA = [gA[1], gA[2]]
print("Clé Publique A: "+str(clePubliqueA))
print("Clé Privée A: "+str(clePriveeA))


Clé Publique A: [158532409092074554599591490678209506821, 97134693687843358162447667011672828645136940852474428829267447741925992387917]
Clé Privée A: [49746300444422200018197235808470484564422296802564297559498529944352406508233, 97134693687843358162447667011672828645136940852474428829267447741925992387917]


In [772]:
#CA
gCA = generateurCle(10**56, 10**57)
clePubliqueCA = [gCA[0], gCA[2]]
clePriveeCA = [gCA[1], gCA[2]]
print("Clé Publique CA: "+str(clePubliqueCA))
print("Clé Privée CA: "+str(clePriveeCA))


Clé Publique CA: [218028488898620386948170921176771082063777384001255247893, 283200925391339198246456136397682771671681666178974909419559311834163898960454394014426197054851260889095732898237]
Clé Privée CA: [102331442368422223369415597557442955464461267863565728509509579099536952036132019111559285531601167239891305178977, 283200925391339198246456136397682771671681666178974909419559311834163898960454394014426197054851260889095732898237]


In [773]:
#Envoie le message au CA
messageA=chiffrement_Asymetrique_Signe(int(clePubliqueA[0]),clePriveeA,clePubliqueCA)

Empreinte: 2848707504997519302384805801290641578297722266561823378148358597769867638886066566103185790925684227660684372646814717802321933341406025863994585971503416
Empreinte Chiffré: 33389934000374791865915488042005048061578194629315864487304192778493762938778509712020183809369413882399400481730207339694653218886154766163562689088319595493528480300883012368872096203147238932854845602278982757198833926082319405239959039480863695285368830802434063311230214837272234025535833599053735876393786776716455778989409051000961450723817015930038421477452255322799488456584050225271113577278244682932558289435460612319912073362569206074384228325930594577303194776664868806510492357858823247583623643779003441373029833089754532685326890242521136733331936847509747096122543973834542110610938601949685578066226181717999177203455060209155297023015110679719479074279783392180439178482658216386993477757159783008678950550423678656447587855511345251158531739739267811478265848126065832256793095761210058292447241

In [774]:
#Vérifie la signature
if(int(verif_signature(messageA[0], messageA[1], clePriveeCA, clePubliqueA)) == clePubliqueA[0]):
    CAClePubliqeA = chiffrer(clePubliqueA[0], clePriveeCA[0], clePriveeCA[1])
    print("Certificat de Alice: "+str(CAClePubliqeA))
else:
    print("Le message a été modifié")


Message déchiffré: 158532409092074554599591490678209506821333899340003747918659154880420050480615781946293158644873041927784937629387785097120201838093694138823994004817302073396946532188861547661635626890883195954935284803008830123688720962031472389328548456022789827571988339260823194052399590394808636952853688308024340633112302148372722340255358335990537358763937867767164557789894090510009614507238170159300384214774522553227994884565840502252711135772782446829325582894354606123199120733625692060743842283259305945773031947766648688065104923578588232475836236437790034413730298330897545326853268902425211367333319368475097470961225439738345421106109386019496855780662261817179991772034550602091552970230151106797194790742797833921804391784826582163869934777571597830086789505504236786564475878555113452511585317397392678114782658481260658322567930957612100582924472413902132551893853017729115210581834447017088613674555981411971752426883434957547311330213355484236468273064023781489329629357584

In [775]:
#Bob
gB = generateurCle(10**39, 10**40)
clePubliqueB = [gB[0], gB[2]]
clePriveeB = [gB[1], gB[2]]

print("Clé Publique de Bob: "+str(clePubliqueB))
print("Clé Privée de Bob: "+str(clePriveeB))

Clé Publique de Bob: [2322249491140717884633842950867886119229, 28229235115081525176947490054988196437946781607755182896700196567416248281921857]
Clé Privée de Bob: [14419147505766356023347196975163305580269269268486373590771045111132490449159469, 28229235115081525176947490054988196437946781607755182896700196567416248281921857]


In [776]:
#Bob vérifie le certificat d'Alice
print("Vérification de la clé publique Alice par Bob")
verifCA=dechiffrer(CAClePubliqeA,clePubliqueCA[0],clePubliqueCA[1])
if(verifCA==clePubliqueA[0]):
    print("La clé publique d'Alice est valide: "+str(verifCA))
else:
    print("Le message a été modifié")

Vérification de la clé publique Alice par Bob
La clé publique d'Alice est valide: 158532409092074554599591490678209506821


In [777]:
# Le message envoyé par Alice
# Le message peut être modifier mais seulement par un autre Integer
messagez = 1435263726352725355365241625326643625185635625635624365265736553536538
print("Le message d'origine de Alice est "+str(messagez))

# Alice chiffre son message
messageCd = chiffrement_Asymetrique_Signe(messagez, clePriveeA, clePubliqueB)

# Bob déchiffre le message d'Alice et vérifie s'il n'y a pas eu de modification
messageDk = verif_signature(
    messageCd[0], messageCd[1], clePriveeB, clePubliqueA)

if(messageDk == -1):
    print("Le message a été modifier")
else:
    print("Le message décrypté est "+str(messageDk))


Le message d'origine de Alice est 1435263726352725355365241625326643625185635625635624365265736553536538
Empreinte: 2848706687633510600576908899941604600646912809140450887991649371245494616730760613400516564373609146429074552439710037523568653121679717901811822237923430
Empreinte Chiffré: 33389934000374791865915488042005048061578194629315864487304192778493762938778509712020183809369413882399400481730207339694653218886154766163562689088319595493528480300883012368872096203147238932854845602278982757198833926082319405247524078636695995264486129074828724543865320637147232719245586279227648888796175195241470142494809813706649743295125877076403260085486533423420159869314634215735387384869840947094770083993009973064443281580803868432759081570723185616201024964262708530415050401160956918002377020846106037331521006838721549368685642641526962807122818548593357377834144960978694245841700537030737852741312519737479499335744620786309490509312021164848201293599699561874483059540765553370097897901981